In [1]:
from os import listdir

def read_file(fn):
    datapoints = list()
    lastp = lastm = None
    for l in open(fn):
        arr = l.split(" ")
        ctime = float(arr[1])
        if ctime == -1:
            continue
        if arr[0] == "p":
            lastp = arr
        if arr[0] == "m":
            lastm = arr
        if lastm and lastp and abs(float(lastm[1])-float(lastp[1]))<0.03:
            datapoints.append([float(x) for x in lastp[1:]+lastm[2:]])
    inputs=list()
    outputs=list()
    if len(datapoints)>1:
        for i,d in enumerate(datapoints[1:]):
            if i==0:
                continue
            #print(datapoints[i])
            #print(d)
            #print("---")
            if abs(d[0]-datapoints[i][0] - 0.1)<0.05:
                inputs.append(datapoints[i-1][1:-4] + datapoints[i][1:])
                # transform outputs into deltas
                outputs.append([result-start for (result, start) in zip(d[1:8],datapoints[i][1:8])])
                if outputs[-1][-1]>15:
                    print(d[1:8])
                    print(datapoints[i][1:8])

    return inputs, outputs

In [2]:
import numpy as np

ins = list()
outs = list()

for fn in listdir("raw/good"):
    ffn="raw/good/"+fn
    print(ffn)
    i,o = read_file(ffn)
    ins+=i
    outs+=o
for fn in listdir("raw/manual/"):
    ffn="raw/manual/"+fn
    print(ffn)
    i,o = read_file(ffn)
    ins+=i
    outs+=o
    





raw/good/coords-2019-5-29-16-48-40.txt
[0.6166287375722932, 0.20703949106010078, -0.4595463732612331, 0.604748412258812, -96.063914, 182.957699, -995.656336]
[0.6182501516376684, 0.20117099796991333, -0.4563298516833082, 0.6075028837520084, -97.041789, 184.829208, -1013.008333]
raw/good/coords-2019-5-29-16-17-5.txt
raw/good/coords-2019-5-29-16-9-46.txt
raw/good/coords-2019-5-29-16-48-2.txt
[0.6207429016911913, 0.20762863280250207, -0.4615252131268397, 0.5988010478852241, -86.622843, 174.248154, -1005.282454]
[0.6157473101849492, 0.20834276963624446, -0.4551254148651092, 0.6085308759001363, -88.091356, 176.072826, -1020.454659]
[0.6138063619741978, 0.2143505804938703, -0.4557756604219752, 0.6079176970402362, -93.13703, 179.285656, -994.248057]
[0.6179010843168994, 0.2089488969625828, -0.45580313281268864, 0.6056252521610364, -94.277137, 181.266333, -1012.997424]
raw/good/coords-2019-5-29-16-52-3.txt
[0.6683520778751272, 0.1394134215849763, -0.48744181515190604, 0.5443066791332234, 48.67

In [3]:
len(ins[-1])

18

In [4]:
# Input "normalization"
for l in ins:
    l[4]/=1000.0
    l[5]/=1000.0
    l[6]/=1000.0
    l[7+4]/=1000.0
    l[7+5]/=1000.0
    l[7+6]/=1000.0
    l[7+7]/=1000.0
    l[7+8]/=1000.0
    l[7+9]/=1000.0
    l[7+10]/=1000.0



In [5]:
# Output "normalization"
for l in outs:
    l[0]*=100.0
    l[1]*=100.0
    l[2]*=100.0
    l[3]*=100.0
    l[4]/=10.0
    l[5]/=10.0
    l[6]/=10.0



In [6]:
from pprint import pprint
print(len(ins), len(outs))

6400 6400


In [7]:
outputs = np.array([[o[4],o[5],o[6]] for o in outs])
inputs = np.array(ins)


In [8]:
np.amin(outputs,0)

array([-0.5748909, -0.5498622, -4.8350689])

In [9]:
from keras.callbacks import Callback

class NBatchLogger(Callback):
    """
    A Logger that log average performance per `display` steps.
    """
    def __init__(self, display):
        self.step = 0
        self.display = display
        self.metric_cache = {}

    def on_batch_end(self, batch, logs={}):
        self.step += 1
        for k in self.params['metrics']:
            if k in logs:
                self.metric_cache[k] = self.metric_cache.get(k, 0) + logs[k]
        if self.step % self.display == 0:
            metrics_log = ''
            for (k, v) in self.metric_cache.items():
                val = v / self.display
                if abs(val) > 1e-3:
                    metrics_log += ' - %s: %.4f' % (k, val)
                else:
                    metrics_log += ' - %s: %.4e' % (k, val)
            print('step: {}/{} ... {}'.format(self.step,
                                          self.params['epochs'],
                                          metrics_log))
            self.metric_cache.clear()

Using TensorFlow backend.
/home/yves/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/yves/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/yves/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/yves/.local/lib/python3.6/site-packages/tensorflow/python/framewo

In [12]:
import warnings
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)

import tensorflow as tf

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.optimizers import Adam, SGD
from keras import backend as K
from keras.layers import LeakyReLU


model = Sequential()
model.add(Dense(160, activation='tanh', input_shape=(18,), use_bias=True))
model.add(Dropout(0.4))
model.add(Dense(160, activation='tanh', use_bias=True))
#model.add(Dense(6000, activation='sigmoid', input_shape=(32,)))
#model.add(Dense(160, activation='relu', input_shape=(32,)))
#model.add(Dense(80, activation='sigmoid'))
#model.add(Dense(40, activation='relu'))
#model.add(Dense(20, activation='linear'))
model.add(Dense(3, activation='linear'))

model.summary()
#opt = SGD(lr=0.08, decay=1e-6, momentum=0.9, nesterov=True)
opt = Adam(lr=0.004, amsgrad=True)
model.compile(loss='mean_squared_error',
  optimizer=opt,
  metrics=['accuracy'])

model.fit(inputs, outputs,
    batch_size=6400,
    epochs=2000,
    verbose=0,
    shuffle=True,
    validation_split=0.0,
    callbacks=[NBatchLogger(display=100)])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 160)               3040      
_________________________________________________________________
dropout_2 (Dropout)          (None, 160)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 160)               25760     
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 483       
Total params: 29,283
Trainable params: 29,283
Non-trainable params: 0
_________________________________________________________________
step: 100/2000 ...  - loss: 0.0975 - acc: 0.3755
step: 200/2000 ...  - loss: 0.0739 - acc: 0.4263
step: 300/2000 ...  - loss: 0.0739 - acc: 0.4289
step: 400/2000 ...  - loss: 0.0737 - acc: 0.4322
step: 500/2000 ...  - loss: 0.0735 - acc: 0.4423
step: 600/2000 ...  - los

In [27]:
model.fit(inputs, outputs,
    batch_size=6400,
    epochs=50000,
    verbose=0,
    shuffle=True,
    validation_split=0.0,
    callbacks=[NBatchLogger(display=100)])

step: 100/50000 ...  - loss: 0.0489 - acc: 0.5887
step: 200/50000 ...  - loss: 0.0491 - acc: 0.5874
step: 300/50000 ...  - loss: 0.0493 - acc: 0.5867
step: 400/50000 ...  - loss: 0.0491 - acc: 0.5870
step: 500/50000 ...  - loss: 0.0493 - acc: 0.5861
step: 600/50000 ...  - loss: 0.0492 - acc: 0.5868
step: 700/50000 ...  - loss: 0.0490 - acc: 0.5873
step: 800/50000 ...  - loss: 0.0493 - acc: 0.5861
step: 900/50000 ...  - loss: 0.0490 - acc: 0.5871
step: 1000/50000 ...  - loss: 0.0490 - acc: 0.5879
step: 1100/50000 ...  - loss: 0.0492 - acc: 0.5871
step: 1200/50000 ...  - loss: 0.0492 - acc: 0.5871
step: 1300/50000 ...  - loss: 0.0490 - acc: 0.5882
step: 1400/50000 ...  - loss: 0.0492 - acc: 0.5864
step: 1500/50000 ...  - loss: 0.0490 - acc: 0.5875
step: 1600/50000 ...  - loss: 0.0491 - acc: 0.5877
step: 1700/50000 ...  - loss: 0.0491 - acc: 0.5862
step: 1800/50000 ...  - loss: 0.0490 - acc: 0.5869
step: 1900/50000 ...  - loss: 0.0491 - acc: 0.5864
step: 2000/50000 ...  - loss: 0.0490 - a

step: 16100/50000 ...  - loss: 0.0478 - acc: 0.5898
step: 16200/50000 ...  - loss: 0.0480 - acc: 0.5882
step: 16300/50000 ...  - loss: 0.0477 - acc: 0.5916
step: 16400/50000 ...  - loss: 0.0478 - acc: 0.5898
step: 16500/50000 ...  - loss: 0.0477 - acc: 0.5900
step: 16600/50000 ...  - loss: 0.0478 - acc: 0.5902
step: 16700/50000 ...  - loss: 0.0477 - acc: 0.5900
step: 16800/50000 ...  - loss: 0.0475 - acc: 0.5917
step: 16900/50000 ...  - loss: 0.0477 - acc: 0.5906
step: 17000/50000 ...  - loss: 0.0478 - acc: 0.5897
step: 17100/50000 ...  - loss: 0.0477 - acc: 0.5898
step: 17200/50000 ...  - loss: 0.0476 - acc: 0.5894
step: 17300/50000 ...  - loss: 0.0474 - acc: 0.5917
step: 17400/50000 ...  - loss: 0.0476 - acc: 0.5907
step: 17500/50000 ...  - loss: 0.0477 - acc: 0.5897
step: 17600/50000 ...  - loss: 0.0475 - acc: 0.5916
step: 17700/50000 ...  - loss: 0.0477 - acc: 0.5903
step: 17800/50000 ...  - loss: 0.0475 - acc: 0.5915
step: 17900/50000 ...  - loss: 0.0474 - acc: 0.5907
step: 18000/

step: 31900/50000 ...  - loss: 0.0463 - acc: 0.5951
step: 32000/50000 ...  - loss: 0.0464 - acc: 0.5950
step: 32100/50000 ...  - loss: 0.0463 - acc: 0.5955
step: 32200/50000 ...  - loss: 0.0464 - acc: 0.5941
step: 32300/50000 ...  - loss: 0.0464 - acc: 0.5941
step: 32400/50000 ...  - loss: 0.0463 - acc: 0.5947
step: 32500/50000 ...  - loss: 0.0462 - acc: 0.5959
step: 32600/50000 ...  - loss: 0.0462 - acc: 0.5952
step: 32700/50000 ...  - loss: 0.0462 - acc: 0.5952
step: 32800/50000 ...  - loss: 0.0461 - acc: 0.5957
step: 32900/50000 ...  - loss: 0.0464 - acc: 0.5941
step: 33000/50000 ...  - loss: 0.0459 - acc: 0.5972
step: 33100/50000 ...  - loss: 0.0461 - acc: 0.5955
step: 33200/50000 ...  - loss: 0.0463 - acc: 0.5951
step: 33300/50000 ...  - loss: 0.0463 - acc: 0.5948
step: 33400/50000 ...  - loss: 0.0460 - acc: 0.5963
step: 33500/50000 ...  - loss: 0.0460 - acc: 0.5959
step: 33600/50000 ...  - loss: 0.0459 - acc: 0.5963
step: 33700/50000 ...  - loss: 0.0461 - acc: 0.5953
step: 33800/

step: 47700/50000 ...  - loss: 0.0449 - acc: 0.5994
step: 47800/50000 ...  - loss: 0.0449 - acc: 0.5987
step: 47900/50000 ...  - loss: 0.0449 - acc: 0.5984
step: 48000/50000 ...  - loss: 0.0449 - acc: 0.5985
step: 48100/50000 ...  - loss: 0.0448 - acc: 0.5995
step: 48200/50000 ...  - loss: 0.0449 - acc: 0.5993
step: 48300/50000 ...  - loss: 0.0447 - acc: 0.6002
step: 48400/50000 ...  - loss: 0.0447 - acc: 0.5998
step: 48500/50000 ...  - loss: 0.0449 - acc: 0.5984
step: 48600/50000 ...  - loss: 0.0447 - acc: 0.5996
step: 48700/50000 ...  - loss: 0.0446 - acc: 0.6003
step: 48800/50000 ...  - loss: 0.0449 - acc: 0.5984
step: 48900/50000 ...  - loss: 0.0450 - acc: 0.5984
step: 49000/50000 ...  - loss: 0.0448 - acc: 0.5993
step: 49100/50000 ...  - loss: 0.0449 - acc: 0.5986
step: 49200/50000 ...  - loss: 0.0447 - acc: 0.5994
step: 49300/50000 ...  - loss: 0.0447 - acc: 0.6002
step: 49400/50000 ...  - loss: 0.0449 - acc: 0.5981
step: 49500/50000 ...  - loss: 0.0447 - acc: 0.6001
step: 49600/

In [26]:

#score = model.evaluate(x_test, y_test, verbose=0)a
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])

ind=535
rx,ry,rz = outputs[ind]
x,y,z, = model.predict(np.array([inputs[ind]]))[0]

print(x,y,z)
print(rx,ry,rz)
#print(((rx-x)**2+(ry-y)**2+(rz-z)**2))
#print(inputs[10])
#print(outputs[10])

0.084226996 -0.076150835 -0.119199276
0.08166539999999997 -0.14852280000000065 0.4672567999999956


In [11]:
np.set_printoptions(suppress=True)
print(inputs[0])

[    0.63964521     0.16337221    -0.47358871     0.58298998
   -96.314989     190.194505   -1002.692322       0.
     0.          -300.             0.        ]
